In [5]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

frescos = {
    "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    # "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    # "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    # "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    # "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    # "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    # "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    # "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    # "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo








Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=2
Category: 141 Name: 141 Image: 141 Price: 141 Stock 141
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=3
Category: 173 Name: 173 Image: 173 Price: 173 Stock 173
Frutas https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=4
No hay más ventanas
Carne https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=0
Category: 221 Name: 221 Image: 221 Price: 221 Stock 221
Carne https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=1
Category: 269 Name: 269 Image: 269 Price: 269 Stock 269
Carne https://www.alcampo.es/compra-onl

,Category,Name,Price,Stock,Image
0,Frutas,Fresón Leyre ALCAMPO PRODUCCIÓN CONTROLADA 400 g.,"(7,22 €/Kg)",Yes,https://www.alcampo.es/media/hc7/hd3/978298965...
1,Frutas,Plátano de Canarias (Indicación Geográfica Pro...,"(3,55 €/Kg)",Yes,https://www.alcampo.es/media/h82/hd5/930404787...
2,Frutas,Plátano Canario extra bolsa,"1,89 €(1,89 €/Kg)",Yes,https://www.alcampo.es/media/h7e/ha2/111832744...
3,Frutas,Banana a granel,"1,59 €(1,59 €/Kg)",Yes,https://www.alcampo.es/media/h3c/h6c/118745949...
4,Frutas,Aguacate malla de 500 g.,"1,79 €(3,58 €/Kg)",Yes,https://www.alcampo.es/media/hee/he4/980467312...
...,...,...,...,...,...
736,Panadería,"Hojaldritos saladitos de sabores surtidos, 12 ...","4,29 €(20,42 €/Kg)",Yes,https://www.alcampo.es/media/h4e/h34/937136537...
737,Panadería,Hogaza 800 g.,"2,87 €(3,58 €/Kg)",No,https://www.alcampo.es/media/h51/hcc/120288490...
738,Panadería,"Triangulo de bácon y queso, 2uds.","1,95 €(8,66 €/Kg)",No,https://www.alcampo.es/media/h8d/h3d/923612161...
739,Panadería,"Empanadilla bocata de carne, 120g.","1,29 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h4e/h11/921102240...


In [6]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

read_file = pd.read_csv ("../data/alcampo.csv")
read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

alcampo = pd.read_csv("../data/alcampo.csv")
print("File saved",today)
alcampo

File saved 2023-03-10 17:34:57.556541


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
35839,Panadería,"Hojaldritos saladitos de sabores surtidos, 12 ...","4,29 €(20,42 €/Kg)",Yes,https://www.alcampo.es/media/h4e/h34/937136537...,Alcampo,2023-03-10 17:34:57.556541
35840,Panadería,Hogaza 800 g.,"2,87 €(3,58 €/Kg)",No,https://www.alcampo.es/media/h51/hcc/120288490...,Alcampo,2023-03-10 17:34:57.556541
35841,Panadería,"Triangulo de bácon y queso, 2uds.","1,95 €(8,66 €/Kg)",No,https://www.alcampo.es/media/h8d/h3d/923612161...,Alcampo,2023-03-10 17:34:57.556541
35842,Panadería,"Empanadilla bocata de carne, 120g.","1,29 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h4e/h11/921102240...,Alcampo,2023-03-10 17:34:57.556541
